## ⭐ Setup

### Packages Handling

In [1]:
%pip install mrcfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import mrcfile
from sklearn.model_selection import train_test_split

### Function

In [3]:
def save_mrc_to_npy(filenames, mrc_dir, save_dir, permissive=False):
  for filename in filenames:
    filepath = os.path.join(mrc_dir, filename)
    print("\rReading mrc:", filepath, end="", flush=True)
    with mrcfile.open(filepath, permissive=permissive) as mrc:
      basename, ext = os.path.splitext(filename)
      assert ext == ".mrc"
      print("\rConverting mrc to npy.", end="", flush=True)
      np.save(os.path.join(save_dir, basename), mrc.data)

## ⭐ Main

In [4]:
# @title Setting directory for mrc files.
from google.colab import drive
drive.mount('/content/drive')

MRC_DIR = "/content/drive/MyDrive/research_xs/dataset/10081/processed_micrographs" # @param {type:"string"}

Mounted at /content/drive


In [16]:
# title Train test split
random_state = 42 # @param {type:"integer"}
test_size = "0.925" # @param {type:"string"}
val_size = "0.019" # @param {type:"string"}
filenames = sorted(os.listdir(MRC_DIR))

test_size = float(test_size)
if test_size and not 0<=test_size<1:
  raise ValueError(f"`test_size` should be between 0.0 and 1.0, got: {test_size}")
val_size = float(val_size)
if val_size and not 0<=val_size<1:
  raise ValueError(f"`val_size` should be between 0.0 and 1.0, got: {val_size}")
train_size = 1 - test_size - val_size
if not 0<train_size<=1:
  raise ValueError(f"`val_size` + `test_size` should be between 0.0 and 1.0, got: {test_size + val_size}")
train_filenames, test_filenames = train_test_split(filenames, test_size=test_size, train_size=train_size, random_state=3)
train_filenames = sorted(train_filenames)
test_filenames = sorted(test_filenames)
val_filenames = []
for filename in filenames:
  if filename not in train_filenames and filename not in test_filenames:
    val_filenames.append(filename)
print("number of files:",
  f" train:\t{len(train_filenames)}",
  f" test:\t{len(test_filenames)}",
  f" val:\t{len(val_filenames)}", sep='\n')

number of files:
 train:	16
 test:	278
 val:	6


In [18]:
%%capture --no-display
# @title Generate np file from mrc files.
# @markdown note that directory for np files should not exist

SAVE_DIR = "/content/drive/MyDrive/research_xs/dataset/10081/processed_micrographs_np_split" # @param {type:"string"}
PERMISSIVE = True # @param {type:"boolean"}

os.mkdir(SAVE_DIR)

os.mkdir(os.path.join(SAVE_DIR, "train"))
save_mrc_to_npy(train_filenames,
                mrc_dir=MRC_DIR,
                save_dir=os.path.join(SAVE_DIR, "train"),
                permissive=PERMISSIVE)
np.savetxt(os.path.join(SAVE_DIR, "train_filenames.txt"), train_filenames, fmt="%s")

os.mkdir(os.path.join(SAVE_DIR, "test"))
save_mrc_to_npy(test_filenames, mrc_dir=MRC_DIR,
                save_dir=os.path.join(SAVE_DIR, "test"),
                permissive=PERMISSIVE)
np.savetxt(os.path.join(SAVE_DIR, "test_filenames.txt"), test_filenames, fmt="%s")

os.mkdir(os.path.join(SAVE_DIR, "val"))
save_mrc_to_npy(val_filenames, mrc_dir=MRC_DIR,
                save_dir=os.path.join(SAVE_DIR, "val"),
                permissive=PERMISSIVE)
np.savetxt(os.path.join(SAVE_DIR, "val_filenames.txt"), val_filenames, fmt="%s")